In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Copa Libertadores y Champions League
### Objetivo 
- 

### Preguntas a responder
- ¿Que resultado es más común?¿Hay una gran diferencia dependiendo de la competicion?
- ¿Que resultado y diferencia de goles es el más comun por cada año?
- ¿Cual es el promedio de goles y de diferencia de goles por año?
- ¿Cual fue el promedio de gol por equipo y por pais?
- Con respecto a la copa libertadores. ¿Contra que paises pierde con más frecuencia los partidos los equipos chilenos?
- ¿En cual competencia gana más el equipo local?
- ¿Cual competición es más dinamica?

#### Para responder todas estas preguntas ocuaperamos los siguientes dataframes.

Primeros sacaremos la informacion de Copa libertadores  y Champions league. Ocuparemos webscrapping.

## Extraccion de informacion Copa Libertadores

In [2]:
LINK_LIB_GR = ["torneo=1048", "torneo=1047", "torneo=1046", "torneo=1045", 
                "torneo=1044", "torneo=1043", "torneo=1074", "torneo=1089"]

LINK_LIB_IDA_VUELTA = ["torneo=1048", "torneo=1047", "torneo=1046", "torneo=1045", 
                        "torneo=1044"]

LINK_LIB_UNICA = ["torneo=1043", "torneo=1074", "torneo=1089", "libertadores"]

DICC_MES = {'Enero': '1', 'Febrero': '2', 'Marzo': '3', 'Abril': '4', 'Mayo': '5', 'Junio': '6', 'Julio': '7', 'Agosto': '8', 
            'Septiembre': '9', 'Octubre': '10', 'Noviembre': '11', 'Diciembre': '12'}

DICT_EQUIPOS = {}

LISTA_RONDAS = ["Final", "Final", "Semifinal", "Semifinal", "Cuartos", "Cuartos", "Octavos", "Octavos"]

def revisar_equipo(l, equipo):
    for i in l:
        eq = i.replace(",", "")
        eq = eq.replace(".", "")
        eq = eq.replace(" ", "")
        if eq[:3].upper() == equipo.upper():
            return i
    return "Dep.Cali"

def im_a_equipo(l):
    paises = []
    for i in l:
        im = i.find("img")
        im = im.get("src")
        im = im.split("/")[2].split(".")[0]
        paises.append(im)
    return paises

locales = []
visitas = []
gol_locales = []
gol_visitas = []
ganador = []
goles = []
resultados = []
dif_goles = []
dias = []
anos = []
fases = []

Una vez ya definida nuestras respectivas variables y funciones. Vamos a definir los partdios de grupo.

In [3]:
ano = 2014
for num in LINK_LIB_GR:
    url = f"https://www.promiedos.com.ar/{num}"
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    lista_grupos = sopa.find_all('div', class_ = 'grupo')
    for data in lista_grupos:
        equipos = data.find('table', id='posiciones')
        equipos = equipos.find_all('td', align = 'left')
        imagen = im_a_equipo(equipos)
        equipos = [i.text.replace(" ", "") for i in equipos]
        partidos = data.find_all('div', id = 'fixgrupo')

        for i in range(4):
            DICT_EQUIPOS[equipos[i]] = imagen[i]

        for datos in partidos:
            horario = datos.find_all('div', class_ = 'diahoragr')
            res = datos.find_all('div', class_ = 'grtr')

            for j in range(2):
                eq1 = res[j].find('div', class_ = 'greq1')
                eq1 = revisar_equipo(equipos, eq1.text)
                locales.append(eq1)
                eq2 = res[j].find('div', class_ = 'greq2')
                eq2 = revisar_equipo(equipos, eq2.text)
                visitas.append(eq2)

                resultado = res[j].find('div', class_ = 'grres4')
                resultado = resultado.text
                resultado = resultado.split('-')
                res1 = int(resultado[0])
                res2 = int(resultado[1])
                gol_locales.append(res1)
                gol_visitas.append(res2)
                resultados.append(str(res1) + "-" + str(res2))

                if res1 == res2:
                    ganador.append("Empate")
                    dif_goles.append(0)

                elif int(res1) > int(res2):
                    ganador.append("Local")
                    dif_goles.append(int(res1) - int(res2))

                elif int(res1) < int(res2):
                    ganador.append("Visita")
                    dif_goles.append(int(res2) - int(res1))

                goles.append(int(res1) + int(res2))
                anos.append(ano)
                dia = horario[j].text
                dia = dia.replace(",", "") ; dia = dia.split(" ")
                dia = dia[3]
                dias.append(dia)
                fases.append("Grupos")

    ano += 1


Añadimos las libertadores con final unica

In [4]:
for num in LINK_LIB_IDA_VUELTA:
    url = f"https://www.promiedos.com.ar/{num}"
    cont = 0
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    lista_fases = sopa.find_all('div', id='partoff')
    for data in lista_fases:
        lista_partidos = data.find_all('div', id='fixturein')
        fase = LISTA_RONDAS[cont]
        for datos in lista_partidos:
            equ = datos.find_all('td', class_='game-t1')
            equ1 = equ[1].text ; equ2 = equ[2].text
            locales.append(equ1)
            visitas.append(equ2)

            res1 = datos.find_all('td', class_='game-r1')
            res1 = (res1[1].text)[0]
            gol_locales.append(int(res1))
            res2 = datos.find_all('td', class_='game-r2')
            res2 = (res2[1].text)[0]
            gol_visitas.append(int(res2))

            if int(res1) == int(res2):
                ganador.append("Empate")
                dif_goles.append(0)

            elif int(res1) > int(res2):
                ganador.append("Local")
                dif_goles.append(int(res1) - int(res2))

            elif int(res1) < int(res2):
                ganador.append("Visita")
                dif_goles.append(int(res2) - int(res1))

            goles.append(int(res1) + int(res2))
            resultados.append(res1 + "-" + res2)


            dia = datos.find_all('tr', class_='diapart')
            dia = dia[0].text
            dia = dia.split(" ")
            anos.append(dia[4])
            ano = dia[4]
            dia = dia[1] + "/"  + DICC_MES[dia[3]]
            dias.append(dia)

            fases.append(fase)

        cont += 1

In [5]:
for num in LINK_LIB_UNICA:
    url = f"https://www.promiedos.com.ar/{num}"
    cont = 2
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    lista_fases = sopa.find_all('div', id='partoff')
    for data in lista_fases:
        lista_partidos = data.find_all('div', id='fixturein')
        fase = LISTA_RONDAS[cont]
        for datos in lista_partidos:
            equ = datos.find_all('td', class_='game-t1')
            equ1 = equ[1].text ; equ2 = equ[2].text
            locales.append(equ1)
            visitas.append(equ2)

            res1 = datos.find_all('td', class_='game-r1')
            res1 = (res1[1].text)[0]
            gol_locales.append(int(res1))
            res2 = datos.find_all('td', class_='game-r2')
            res2 = (res2[1].text)[0]
            gol_visitas.append(int(res2))

            if int(res1) == int(res2):
                ganador.append("Empate")
                dif_goles.append(0)

            elif int(res1) > int(res2):
                ganador.append("Local")
                dif_goles.append(int(res1) - int(res2))

            elif int(res1) < int(res2):
                ganador.append("Visita")
                dif_goles.append(int(res2) - int(res1))

            goles.append(int(res1) + int(res2))
            resultados.append(res1 + "-" + res2)


            dia = datos.find_all('tr', class_='diapart')
            dia = dia[0].text
            dia = dia.split(" ")
            if len(dia) < 5:
                anos.append("2022")
            else:
                anos.append(dia[4])
            dia = dia[1] + "/"  + DICC_MES[dia[3]]
            dias.append(dia)

            fases.append(fase)

        cont += 1
        if cont == 8:
            break

In [6]:
url = "https://www.promiedos.com.ar/libertadores=faseprevia"
page = rq.get(url)
page = bs(page.content, 'html.parser')
tabla = page.find('div', id='tablapts')
tabla = tabla.find_all('div', class_='grupo')
cont = 1
for data in tabla:
    l_equipos = []
    equipo = data.find('table', class_=f'tablesorter{cont}')
    equipo = equipo.find_all('tr')
    equipo.pop(0)
    imgs = im_a_equipo(equipo)
    for nom in equipo:
        nom = nom.find('td', align='left')
        l_equipos.append(nom.text.replace(' ', ''))
    for i in range(4):
        DICT_EQUIPOS[l_equipos[i]] = imgs[i]
    partidos = data.find_all('div', id = 'fixgrupo')
    cont += 1
    for datos in partidos:
        horario = datos.find_all('div', class_ = 'diahoragr')
        res = datos.find_all('div', class_ = 'grtr')

        for j in range(2):
            eq1 = res[j].find('div', class_ = 'greq1')
            eq1 = revisar_equipo(l_equipos, eq1.text)
            locales.append(eq1)
            eq2 = res[j].find('div', class_ = 'greq2')
            eq2 = revisar_equipo(l_equipos, eq2.text)
            visitas.append(eq2)

            resultado = res[j].find('div', class_ = 'grres4')
            resultado = resultado.text
            resultado = resultado.split('-')
            res1 = int(resultado[0])
            res2 = int(resultado[1])
            gol_locales.append(res1)
            gol_visitas.append(res2)
            resultados.append(str(res1) + "-" + str(res2))

            if res1 == res2:
                ganador.append("Empate")
                dif_goles.append(0)

            elif int(res1) > int(res2):
                ganador.append("Local")
                dif_goles.append(int(res1) - int(res2))

            elif int(res1) < int(res2):
                ganador.append("Visita")
                dif_goles.append(int(res2) - int(res1))

            goles.append(int(res1) + int(res2))
            anos.append(ano)
            dia = horario[j].text
            dia = dia.replace(",", "") ; dia = dia.split(" ")
            dia = dia[3]
            dias.append(dia)
            fases.append("Grupos")


In [7]:
locales = [i.replace(" ", "") for i in locales]
visitas = [i.replace(" ", "") for i in visitas]

In [8]:
paises_local = [DICT_EQUIPOS[i] for i in locales]
paises_visitas = [DICT_EQUIPOS[i] for i in visitas]

In [9]:
df_libertadores = pd.DataFrame()
df_libertadores["Local"] = locales
df_libertadores["Pais_local"] = paises_local
df_libertadores["Visita"] = visitas
df_libertadores["Pais_visita"] = paises_visitas
df_libertadores["Gol_local"] = gol_locales
df_libertadores["Gol_visita"] = gol_visitas
df_libertadores["Ganador"] = ganador
df_libertadores["Resultado"] = resultados
df_libertadores["Año"] = anos
df_libertadores["Fase"] = fases
df_libertadores["Goles"] = goles
df_libertadores["Diferencia_goles"] = dif_goles
df_libertadores["Dia"] = dias
df_libertadores

,Local,Pais_local,Visita,Pais_visita,Gol_local,Gol_visita,Ganador,Resultado,Año,Fase,Goles,Diferencia_goles,Dia
0,Universitario(P),peru,Velez,argentina,0,1,Visita,0-1,2014,Grupos,1,1,11/2
1,Paranaense,brasil,TheStrongest,bolivia,1,0,Local,1-0,2014,Grupos,1,1,13/2
2,TheStrongest,bolivia,Universitario(P),peru,1,0,Local,1-0,2014,Grupos,1,1,20/2
3,Velez,argentina,Paranaense,brasil,2,0,Local,2-0,2014,Grupos,2,2,25/2
4,TheStrongest,bolivia,Velez,argentina,2,0,Local,2-0,2014,Grupos,2,2,11/3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107,Sp.Cristal,peru,U.Catolica,chile,1,1,Empate,1-1,2018,Grupos,2,0,4/5
1108,Flamengo,brasil,U.Catolica,chile,3,0,Local,3-0,2018,Grupos,3,3,17/5
1109,Sp.Cristal,peru,Talleres(C),argentina,0,0,Empate,0-0,2018,Grupos,0,0,17/5
1110,Flamengo,brasil,Sp.Cristal,peru,2,1,Local,2-1,2018,Grupos,3,1,24/5


## Extraccion de informacion Champion League

In [10]:
LINK_GRUP_CHMPS = ["torneo=1068", "torneo=1067", "torneo=1066", "torneo=1065", "torneo=1073", "torneo=1086", 
                    "torneo=1104", "championsleague"]
LINK_CHMPS =    ["torneo=1068", "torneo=1067", "torneo=1066", "torneo=1065", "torneo=1073",  "torneo=1086", 
                    "torneo=1104"]

DICC_MES = {'Enero': '1', 'Febrero': '2', 'Marzo': '3', 'Abril': '4', 'Mayo': '5', 'Junio': '6', 'Julio': '7', 'Agosto': '8', 
            'Septiembre': '9', 'Octubre': '10', 'Noviembre': '11', 'Diciembre': '12'}
            
DICT_EQUIPOS = {}

LISTA_RONDAS = ["Final", "Final", "Semifinal", "Semifinal", "Cuartos", "Cuartos", "Octavos", "Octavos"]

def revisar_equipo(l, equipo):
    for i in l:
        eq = i.replace(",", "")
        eq = eq.replace(".", "")
        eq = eq.replace(" ", "")
        if eq[:3].upper() == equipo.upper():
            return i

def im_a_equipo(l):
    paises = []
    for i in l:
        im = i.find("img")
        im = im.get("src")
        im = im.split("/")[2].split(".")
        if im[1] == "png":
            im = "test"
        else:
            im = im[0]
            
        paises.append(im)
    return paises

locales = []
visitas = []
gol_locales = []
gol_visitas = []
ganador = []
goles = []
resultados = []
dif_goles = []
dias = []
anos = []
fases = []

In [11]:
ano = 2015
for num in LINK_GRUP_CHMPS:
    url = f"https://www.promiedos.com.ar/{num}"
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    lista_grupos = sopa.find_all('div', class_ = 'grupo')
    for data in lista_grupos:
        equipos = data.find('table', id='posiciones')
        equipos = equipos.find_all('td', align = 'left')
        imagen = im_a_equipo(equipos)
        equipos = [i.text.replace(" ", "") for i in equipos]
        partidos = data.find_all('div', id = 'fixgrupo')

        for i in range(4):
            DICT_EQUIPOS[equipos[i]] = imagen[i]

        for datos in partidos:
            horario = datos.find_all('div', class_ = 'diahoragr')
            res = datos.find_all('div', class_ = 'grtr')

            for j in range(2):
                eq1 = res[j].find('div', class_ = 'greq1')
                eq1 = revisar_equipo(equipos, eq1.text)
                locales.append(eq1)
                eq2 = res[j].find('div', class_ = 'greq2')
                eq2 = revisar_equipo(equipos, eq2.text)
                visitas.append(eq2)

                resultado = res[j].find('div', class_ = 'grres4')
                resultado = resultado.text
                resultado = resultado.split('-')
                res1 = int(resultado[0])
                res2 = int(resultado[1])
                gol_locales.append(res1)
                gol_visitas.append(res2)
                resultados.append(str(res1) + "-" + str(res2))

                if res1 == res2:
                    ganador.append("Empate")
                    dif_goles.append(0)

                elif int(res1) > int(res2):
                    ganador.append("Local")
                    dif_goles.append(int(res1) - int(res2))

                elif int(res1) < int(res2):
                    ganador.append("Visita")
                    dif_goles.append(int(res2) - int(res1))

                goles.append(int(res1) + int(res2))
                anos.append(ano)
                dia = horario[j].text
                dia = dia.replace(",", "") ; dia = dia.split(" ")
                dia = dia[3]
                dias.append(dia)
                fases.append("Grupos")

    ano += 1

Agregamos las fases finales

Excepto las fases finales de 2020, ya que estas tienen otro formato, del 2020 solo agregamos la final

In [12]:
for num in LINK_CHMPS:
    url = f"https://www.promiedos.com.ar/{num}"
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    final_unica = sopa.find_all('div', id='partoff1')
    equ = final_unica[0].find_all('td', class_='game-t1')
    equ = final_unica[0].find_all('td', class_='game-t1')
    equ1 = equ[1].text.replace(' ', '') ; equ2 = equ[2].text.replace(' ', '')
    locales.append(equ1)
    visitas.append(equ2)
    res1 = final_unica[0].find_all('td', class_='game-r1')
    res1 = (res1[1].text)[0]
    gol_locales.append(int(res1))
    res2 = final_unica[0].find_all('td', class_='game-r2')
    res2 = (res2[1].text)[0]
    gol_visitas.append(int(res2))

    if int(res1) == int(res2):
        ganador.append("Empate")
        dif_goles.append(0)

    elif int(res1) > int(res2):
        ganador.append("Local")
        dif_goles.append(int(res1) - int(res2))

    elif int(res1) < int(res2):
        ganador.append("Visita")
        dif_goles.append(int(res2) - int(res1))

    goles.append(int(res1) + int(res2))
    resultados.append(res1 + "-" + res2)


    dia = final_unica[0].find_all('tr', class_='diapart')
    dia = dia[0].text
    dia = dia.split(" ")
    if len(dia) <= 4:
        dia.append("2022")
    anos.append(dia[4])
    ano = dia[4]
    dia = dia[1] + '/' + DICC_MES[dia[3]]
    dias.append(dia)
    fases.append("Final")

In [13]:
LINK_CHMPS.pop(4)
for num in LINK_CHMPS:
    url = f"https://www.promiedos.com.ar/{num}"
    cont = 2
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    lista_fases = sopa.find_all('div', id='partoff')
    for data in lista_fases:
        lista_partidos = data.find_all('div', id='fixturein')
        fase = LISTA_RONDAS[cont]
        for datos in lista_partidos:
            equ = datos.find_all('td', class_='game-t1')
            equ1 = equ[1].text ; equ2 = equ[2].text
            locales.append(equ1)
            visitas.append(equ2)

            res1 = datos.find_all('td', class_='game-r1')
            res1 = (res1[1].text)[0]
            gol_locales.append(int(res1))
            res2 = datos.find_all('td', class_='game-r2')
            res2 = (res2[1].text)[0]
            gol_visitas.append(int(res2))

            if int(res1) == int(res2):
                ganador.append("Empate")
                dif_goles.append(0)

            elif int(res1) > int(res2):
                ganador.append("Local")
                dif_goles.append(int(res1) - int(res2))

            elif int(res1) < int(res2):
                ganador.append("Visita")
                dif_goles.append(int(res2) - int(res1))

            goles.append(int(res1) + int(res2))
            resultados.append(res1 + "-" + res2)


            dia = datos.find_all('tr', class_='diapart')
            dia = dia[0].text
            dia = dia.split(" ")
            if len(dia) < 5:
                anos.append("2022")
            else:
                anos.append(dia[4])
            dia = dia[1] + "/"  + DICC_MES[dia[3]]
            dias.append(dia)

            fases.append(fase)

        cont += 1
        if cont == 8:
            break

Ahora agregamos la del 2020

In [14]:
url = f"https://www.promiedos.com.ar/torneo=1073"
cont = 4
r = rq.get(url)
sopa = bs(r.content, "html.parser")
final_unica = sopa.find_all('div', id='partoff1')
final_unica.pop(0)
data = final_unica[0].find_all('div', id = 'fixturein')
for datos in data:
    equ = datos.find_all('td', class_='game-t1')
    equ1 = equ[1].text ; equ2 = equ[2].text
    locales.append(equ1)
    visitas.append(equ2)

    res1 = datos.find_all('td', class_='game-r1')
    res1 = (res1[1].text)[0]
    gol_locales.append(int(res1))
    res2 = datos.find_all('td', class_='game-r2')
    res2 = (res2[1].text)[0]
    gol_visitas.append(int(res2))

    if int(res1) == int(res2):
        ganador.append("Empate")
        dif_goles.append(0)

    elif int(res1) > int(res2):
        ganador.append("Local")
        dif_goles.append(int(res1) - int(res2))

    elif int(res1) < int(res2):
        ganador.append("Visita")
        dif_goles.append(int(res2) - int(res1))

    goles.append(int(res1) + int(res2))
    resultados.append(res1 + "-" + res2)


    dia = datos.find_all('tr', class_='diapart')
    dia = dia[0].text
    dia = dia.split(" ")
    anos.append(dia[4])
    ano = dia[4]
    dia = dia[1] + "/"  + DICC_MES[dia[3]]
    dias.append(dia)
    fase = "Semifinal"
    fases.append(fase)

In [15]:
url = f"https://www.promiedos.com.ar/torneo=1073"
cont = 4
r = rq.get(url)
sopa = bs(r.content, "html.parser")
lista_fases = sopa.find_all('div', id='partoff')
for data in lista_fases:
    lista_partidos = data.find_all('div', id='fixturein')
    fase = LISTA_RONDAS[cont]
    for datos in lista_partidos:
        equ = datos.find_all('td', class_='game-t1')
        equ1 = equ[1].text ; equ2 = equ[2].text
        locales.append(equ1)
        visitas.append(equ2)

        res1 = datos.find_all('td', class_='game-r1')
        res1 = (res1[1].text)[0]
        gol_locales.append(int(res1))
        res2 = datos.find_all('td', class_='game-r2')
        res2 = (res2[1].text)[0]
        gol_visitas.append(int(res2))

        if int(res1) == int(res2):
            ganador.append("Empate")
            dif_goles.append(0)

        elif int(res1) > int(res2):
            ganador.append("Local")
            dif_goles.append(int(res1) - int(res2))

        elif int(res1) < int(res2):
            ganador.append("Visita")
            dif_goles.append(int(res2) - int(res1))

        goles.append(int(res1) + int(res2))
        resultados.append(res1 + "-" + res2)


        dia = datos.find_all('tr', class_='diapart')
        dia = dia[0].text
        dia = dia.split(" ")
        if len(dia) < 5:
            anos.append("2022")
        else:
            anos.append(dia[4])
        dia = dia[1] + "/"  + DICC_MES[dia[3]]
        dias.append(dia)

        fases.append(fase)

    cont += 1
    if cont == 8:
        break

In [16]:
locales = [i.replace(" ", "") for i in locales]
visitas = [i.replace(" ", "") for i in visitas]

In [17]:
DICT_EQUIPOS["D.Zagreb"] = "croacia" ; DICT_EQUIPOS["Maccabi"] = "israel" ; DICT_EQUIPOS["Gent"] = "belgica"
DICT_EQUIPOS["Rostov"] = "rusia" ; DICT_EQUIPOS["CSKA"] = "rusia" ; DICT_EQUIPOS["Maribor"] = "estonia"

In [18]:
paises_local = [DICT_EQUIPOS[i] for i in locales]
paises_visitas = [DICT_EQUIPOS[i] for i in visitas]

In [19]:
df_champions = pd.DataFrame()
df_champions["Local"] = locales
df_champions["Pais_local"] = paises_local
df_champions["Visita"] = visitas
df_champions["Pais_visita"] = paises_visitas
df_champions["Gol_local"] = gol_locales
df_champions["Gol_visita"] = gol_visitas
df_champions["Ganador"] = ganador
df_champions["Resultado"] = resultados
df_champions["Año"] = anos
df_champions["Fase"] = fases
df_champions["Goles"] = goles
df_champions["Diferencia_goles"] = dif_goles
df_champions["Dia"] = dias
df_champions

,Local,Pais_local,Visita,Pais_visita,Gol_local,Gol_visita,Ganador,Resultado,Año,Fase,Goles,Diferencia_goles,Dia
0,PSG,francia,Malmo,suecia,2,0,Local,2-0,2015,Grupos,2,2,15/9
1,RealMadrid,espana,Shakhtar,ucrania,4,0,Local,4-0,2015,Grupos,4,4,15/9
2,Malmo,suecia,RealMadrid,espana,0,2,Visita,0-2,2015,Grupos,2,2,30/9
3,Shakhtar,ucrania,PSG,francia,0,3,Visita,0-3,2015,Grupos,3,3,30/9
4,Malmo,suecia,Shakhtar,ucrania,1,0,Local,1-0,2015,Grupos,1,1,21/10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,Juventus,italia,Lyon,francia,2,1,Local,2-1,2020,Octavos,3,1,07/8
961,Tottenham,inglaterra,RBLeipzig,alemania,0,1,Visita,0-1,2020,Octavos,1,1,19/2
962,RBLeipzig,alemania,Tottenham,inglaterra,3,0,Local,3-0,2020,Octavos,3,3,10/3
963,Napoli,italia,Barcelona,espana,1,1,Empate,1-1,2020,Octavos,2,0,25/2


¿Que resultado es más común?¿Hay una gran diferencia dependiendo de la competicion?

In [20]:
df_libertadores['Resultado'].value_counts().head()

1-0    129
1-1    128
2-0    110
0-0     93
0-1     91
Name: Resultado, dtype: int64

In [21]:
df_champions['Resultado'].value_counts().head()

1-1    89
2-0    79
2-1    76
1-2    76
1-0    69
Name: Resultado, dtype: int64

In [22]:
df_champions['Resultado'].value_counts().index[0]

'1-1'

¿Que resultado y diferencia de goles es el más comun por cada año?

In [23]:
df_res_año = df_libertadores[['Resultado', 'Año', 'Diferencia_goles']]
years = df_res_año['Año'].unique()
for p in years :
    df_aux = df_res_año[df_res_año['Año']== p]
    comun_a = df_aux['Resultado'].value_counts().index[0]

    df_aux = df_res_año[df_res_año['Año']== p]
    comun_d = df_aux['Diferencia_goles'].value_counts().index[0]
    print(f"En el año {p} El resultado más común fue {comun_a} y la diferencia de goles con mayor frecuencia fue de {comun_d}")

En el año 2014 El resultado más común fue 1-0 y la diferencia de goles con mayor frecuencia fue de 1
En el año 2015 El resultado más común fue 2-0 y la diferencia de goles con mayor frecuencia fue de 1
En el año 2016 El resultado más común fue 1-1 y la diferencia de goles con mayor frecuencia fue de 1
En el año 2017 El resultado más común fue 1-1 y la diferencia de goles con mayor frecuencia fue de 1
En el año 2018 El resultado más común fue 0-0 y la diferencia de goles con mayor frecuencia fue de 1
En el año 2019 El resultado más común fue 1-0 y la diferencia de goles con mayor frecuencia fue de 1
En el año 2020 El resultado más común fue 3-0 y la diferencia de goles con mayor frecuencia fue de 1
En el año 2021 El resultado más común fue 1-1 y la diferencia de goles con mayor frecuencia fue de 1
En el año 2014 El resultado más común fue 1-0 y la diferencia de goles con mayor frecuencia fue de 1
En el año 2015 El resultado más común fue 1-0 y la diferencia de goles con mayor frecuencia

¿Cual es el promedio de goles y de diferencia de goles por año?

In [25]:
df_res_año = df_libertadores[['Goles', 'Año', 'Diferencia_goles']]
years = df_res_año['Año'].unique()
for p in years :
    df_aux = df_res_año[df_res_año['Año']== p]
    comun_g = df_aux['Goles'].mean().round()

    df_aux = df_res_año[df_res_año['Año']== p]
    comun_d = df_aux['Diferencia_goles'].mean().round()
    print(f"En el año {p} El Goles más común fue {comun_g} y la diferencia de goles con mayor frecuencia fue de {comun_d}")

En el año 2014 El Goles más común fue 3.0 y la diferencia de goles con mayor frecuencia fue de 1.0
En el año 2015 El Goles más común fue 3.0 y la diferencia de goles con mayor frecuencia fue de 2.0
En el año 2016 El Goles más común fue 3.0 y la diferencia de goles con mayor frecuencia fue de 1.0
En el año 2017 El Goles más común fue 3.0 y la diferencia de goles con mayor frecuencia fue de 1.0
En el año 2018 El Goles más común fue 2.0 y la diferencia de goles con mayor frecuencia fue de 1.0
En el año 2019 El Goles más común fue 2.0 y la diferencia de goles con mayor frecuencia fue de 1.0
En el año 2020 El Goles más común fue 3.0 y la diferencia de goles con mayor frecuencia fue de 2.0
En el año 2021 El Goles más común fue 3.0 y la diferencia de goles con mayor frecuencia fue de 2.0
En el año 2014 El Goles más común fue 2.0 y la diferencia de goles con mayor frecuencia fue de 1.0
En el año 2015 El Goles más común fue 2.0 y la diferencia de goles con mayor frecuencia fue de 1.0
En el año 